## *PhotoDissociation Region Toolbox Notebooks*
---------------------------------------------------------------

# Example 2: Introduction to ModelSets

PDRT supports a variety of PDR models to be used to fit your data.  

The Wolfire/Kaufman 2020 (WK2020) ModelSet contains constant density models for metallicity $z=1$. There are also a WK2006 z=3 and z=0.1 sets, with limited spectral lines. Models are stored as ratios of intensities as a function of radiation field $I_{FUV}$ and hydrogen nucleus volume density $n$. In some cases, the individual intensity models are also available (but fitting with `LineRatioFit` only uses the ratios). We will continue to expand 2020 model set to include a wider range of lines and metallicities.  

We also have available [KOSMA-tau](https://astro.uni-koeln.de/stutzki/research/kosma-tau)  ModelSets from Markus Rollig. These are both 'clumpy' and non-clumpy' models with varying mass and $R_v$ value and $z=1$. In a non-clumpy model the mass is of the spherical clump. The density (profile) and the mass determine the clump radius and the total $A_V$ to the clump center. In the clumpy model the mass is the total (ensemble) mass of all clumps which are distributed according to a power law with the mass range $[M_{low}, M_{up}]$. 

Any PDR models
can be used if they are stored in the [correct FITS format](https://docs.google.com/document/d/1hPSUip5RK6nPmqrL8iw3qkX69DVLBv0sAuqac18pLT8/edit?usp=sharing). 
 

In [1]:
from pdrtpy.modelset import ModelSet
from pdrtpy import pdrutils as utils

### all_sets() describes all the ModelSets available. 

In [2]:
ModelSet.all_sets().show_in_notebook()

idx,PDR code,name,version,medium,z,mass,description
,,,,,,solMass,
0,Wolfire/Kaufman,wk2006,2006,constant density,1.0,--,Wolfire/Kaufman 2006 constant density models
1,Wolfire/Kaufman,wk2006,2006,constant density,3.0,--,Wolfire/Kaufman 2006 constant density models
2,Wolfire/Kaufman,wk2020,2020,constant density,1.0,--,"Wolfire/Kaufman 2020 constant density models, $A_V$=7, no freeze-out"
3,Wolfire/Kaufman,smc,2006,constant density,0.1,--,Wolfire/Kaufman 2006 limited set of constant density models for Small Magellanic Cloud
4,Wolfire/Kaufman,lmc,2020,constant density,0.5,--,"Wolfire/Kaufman 2020 limited set of constant density models for Large Magellanic Cloud, surface chemistry turned off"
5,KOSMA-tau,kt2013wd01-7,WD01-7,clumpy,1.0,100.0,"KOSMA-tau 2013 models, Weingarter & Draine 2001 $R_V$=3.1, clumpy, maximum clump mass = 100 M$_\odot$"
6,KOSMA-tau,kt2013wd01-7,WD01-7,non-clumpy,1.0,100.0,"KOSMA-tau 2013 models, Weingarter & Draine 2001 $R_V$=3.1, non-clumpy, single clump 100 M$_\odot$"
7,KOSMA-tau,kt2013wd01-21,WD01-21,clumpy,1.0,100.0,"KOSMA-tau 2013 models, Weingarter & Draine 2001 $R_V$=4.0, clumpy, maximum clump mass = 100 M$_\odot$"
8,KOSMA-tau,kt2013wd01-21,WD01-21,non-clumpy,1.0,100.0,"KOSMA-tau 2013 models, Weingarter & Draine 2001 $R_V$=4.0, non-clumpy, single clump 100 M$_\odot$"


### Instantiate the KOSMA-tau 2013 clumpy, 10 $M_\odot$, $R_v$=5.5 models for z=1. 

In [3]:
ms = ModelSet("kt2013wd01-25",medium="clumpy",mass=10,z=1)

In [4]:
ms._supported_ratios

title,ratio label
str50,str18
[O I] 145 $\mu$m/[O I] 63 $\mu$m,OI_145/OI_63
[O I] 145 $\mu$m/[C II] 158 $\mu$m,OI_145/CII_158
[O I] 63 $\mu$m/[C II] 158 $\mu$m,OI_63/CII_158
[C II] 158 $\mu$m/[C I] 609 $\mu$m,CII_158/CI_609
[C I] 370 $\mu$m/[C I] 609 $\mu$m,CI_370/CI_609
...,...
CO(J=13-12)/CO(J=6-5),CO_1312/CO_65
CO(J=14-13)/CO(J=5-4),CO_1413/CO_54
CO(J=14-13)/CO(J=6-5),CO_1413/CO_65


### Show the model intensities and ratios that are available in this ModelSet
`supported_intensities` and `supported_ratios` is an astropy Table, invoke *show_in_notebook()* to see all the rows.  There may be more ratios than lines because not all components of a ratio may be available as separate models.

In [5]:
ms.supported_intensities

title,intensity label
null,
str50,str18
I$_{FIR}$,FIR


Every model has a one-line description. You can query a ModelSet to find which spectral lines (your observations) are supported and which model ratios it has for your lines.

In [6]:
print(ms.description)

mylines =["CII_158","OI_145","FIR","CI_609","CO_65"]
print("There are %d ratios in the %s ModelSet for the lines %s:"%(ms.ratiocount(mylines),ms.name,mylines))
print(ms.model_ratios(mylines))

# If you want to see the actual FITS file names. The root of the path is where your pdrtpy is installed.
print("\nThey are located in:")
for p in ms.find_files(mylines):
    print(p)

KOSMA-tau 2013 models, Weingarter & Draine 2001 $R_V$=5.5, clumpy, maximum clump mass = 10 M$_\odot$, Z=1.0
There are 4 ratios in the kt2013wd01-25 ModelSet for the lines ['CII_158', 'OI_145', 'FIR', 'CI_609', 'CO_65']:
['CII_158/CI_609', 'CII_158/CO_65', 'OI_145/CII_158', 'OI_145+CII_158/FIR']

They are located in:
('CII_158/CI_609', 'kosmatau/2013/WD01-25/clumpy/Z=1/M=10/CII158_CI609.fits')
('CII_158/CO_65', 'kosmatau/2013/WD01-25/clumpy/Z=1/M=10/CII158_CO65.fits')
('OI_145/CII_158', 'kosmatau/2013/WD01-25/clumpy/Z=1/M=10/OI145_CII158.fits')
('OI_145+CII_158/FIR', 'kosmatau/2013/WD01-25/clumpy/Z=1/M=10/OI145+CII158_FIR.fits')


## How to access a single model
This example returns the model for \[C II\] 158 $\mu$m/CO(4-3) intensity ratio. The access key is the tag in the `ratio label` in the supported ratios table above.  The model is returned as a `pdrtpy.measurement.Measurement`

In [7]:
cico = ms.get_model("CII_158/CO_43")
print(f"{cico.id} Min value:{cico.data.min():.1e} Max value:{cico.data.max():.1e}")
print(f"Density units: {cico.header['CUNIT1']}, radiation field units: {cico.header['CUNIT2']}")

CII_158/CO_43 Min value:4.9e-06 Max value:9.7e+08
Density units: 1 / cm3, radiation field units: erg / (cm2 s)


In [8]:
ms.supported_ratios

title,ratio label
str50,str18
[O I] 145 $\mu$m/[O I] 63 $\mu$m,OI_145/OI_63
[O I] 145 $\mu$m/[C II] 158 $\mu$m,OI_145/CII_158
[O I] 63 $\mu$m/[C II] 158 $\mu$m,OI_63/CII_158
[C II] 158 $\mu$m/[C I] 609 $\mu$m,CII_158/CI_609
[C I] 370 $\mu$m/[C I] 609 $\mu$m,CI_370/CI_609
...,...
CO(J=13-12)/CO(J=6-5),CO_1312/CO_65
CO(J=14-13)/CO(J=5-4),CO_1413/CO_54
CO(J=14-13)/CO(J=6-5),CO_1413/CO_65


## Probe the ratio or intensity values in a model
Once you have a model you can use the `get` function to discover the model value at a particular density and radiation field.  You must give the density and radiation field in the units of the model.  For WK2020, this is cm$^{-3}$ and erg cm$^{-2}$ s$^{-1}$.  However, you can use astropy's `Quantity` to convert for you.

In [9]:
n=1234.5
rf=0.6789
cico.get(n,rf)

array([256.03939574])

#### Now suppose you want to use Habing for the radiation field
Use Quantity to convert and pass the value to `get`.

In [10]:
from astropy.units import Quantity
rf=Quantity(2.3,'Habing')
print(f"{rf.to(cico.header['CUNIT2']):0.2e}")
cico.get(n,rf.to(cico.header['CUNIT2']).value)

3.68e-03 erg / (cm2 s)


array([3.32513494])

### How to access multiple models
Suppose you want to find all the models that use some combination of CO(J=1-0), \[C II\] 158 $\mu$m, \[O I\] 145 $\mu$m, and $I_{FIR}$. This example gets both intensity and ratio models, though you can specify `model_type='intensity'` or `model_type='ratio'` to get one or the other.  The models are returns as a dictionary with the keys set to the model IDs.

In [11]:
mods = ms.get_models(["CII_158","OI_145", "CO_10", "FIR"],model_type='both')

#### The returned models are three ratio models and one intensity model

In [12]:
mods.keys()

dict_keys(['FIR', 'CII_158/CO_10', 'OI_145/CII_158', 'OI_145+CII_158/FIR'])

In [13]:
# average value of one of the models
print(mods['CII_158/CO_10'].data.mean())

1901273803.7736897
